In [5]:
import ee
import folium
import ipywidgets as widgets
from IPython.display import display

# Initialize the Earth Engine API.
ee.Initialize()

# Dictionary to hold country-specific parameters.
# Adjust the 'center', 'zoom', and 'geojson' paths as needed.
countries = {
    'mauritania': {
        'center': [20, -10],
        'zoom': 6,
        'geojson': '../static/geojson/mauritania.json'
    },
    'burkinafaso': {
        'center': [12, -1],
        'zoom': 6,
        'geojson': '../static/geojson/burkinafaso.json'
    },
    'chad': {
        'center': [15, 19],
        'zoom': 5,
        'geojson': '../static/geojson/chad.json'
    },
    'mali': {
        'center': [17, -3],
        'zoom': 5,
        'geojson': '../static/geojson/mali.json'
    },
    'niger': {
        'center': [16, 8],
        'zoom': 5,
        'geojson': '../static/geojson/niger.json'
    },
    'senegal': {
        'center': [14, -14],
        'zoom': 6,
        'geojson': '../static/geojson/senegal.json'
    },
    'sudan': {
        'center': [15, 30],
        'zoom': 5,
        'geojson': '../static/geojson/sudan.json'
    }
}

# Function to get a MODIS Land Cover image for a given year.
def get_modis_image(year):
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'
    image = (ee.ImageCollection('MODIS/006/MCD12Q1')
             .filterDate(start_date, end_date)
             .first()
             .select('LC_Type1'))
    return image

# Visualization parameters for the MODIS land cover classification.
vis_params = {
    'min': 0,
    'max': 17,
    'palette': [
        '05450a',  # Evergreen Needleleaf Forests (Class 1)
        '086a10',  # Evergreen Broadleaf Forests (Class 2)
        '54a708',  # Deciduous Needleleaf Forests (Class 3)
        '78d203',  # Deciduous Broadleaf Forests (Class 4)
        '009900',  # Mixed Forests (Class 5)
        'c6b044',  # Closed Shrublands (Class 6)
        'dcd159',  # Open Shrublands (Class 7)
        'dade48',  # Woody Savannas (Class 8)
        'fbff13',  # Savannas (Class 9)
        'b6ff05',  # Grasslands (Class 10)
        '27ff87',  # Permanent Wetlands (Class 11)
        'c24f44',  # Croplands (Class 12)
        'a5a5a5',  # Urban and Built-up Lands (Class 13)
        'ff6d4c',  # Cropland/Natural Vegetation Mosaics (Class 14)
        '69fff8',  # Permanent Snow and Ice (Class 15)
        'f9ffa4',  # Barren (Class 16)
        '1c0dff'   # Water Bodies (Class 17)
    ]
}

# Helper function to add Earth Engine layers to a Folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Function to add a legend to the map with just the class names.
def add_legend(map_object):
    legend_html = '''
     <div style="
         position: fixed; 
         bottom: 50px; left: 50px; 
         width: 240px; 
         border:2px solid grey; 
         z-index:9999; 
         font-size:12px;
         background-color:white;
         opacity: 0.8;
         padding: 10px;
         ">
         <b>Land Cover Legend</b><br>
         <i style="background:#05450a; width:18px; height:18px; float:left; margin-right:8px;"></i>Evergreen Needleleaf Forests<br>
         <i style="background:#086a10; width:18px; height:18px; float:left; margin-right:8px;"></i>Evergreen Broadleaf Forests<br>
         <i style="background:#54a708; width:18px; height:18px; float:left; margin-right:8px;"></i>Deciduous Needleleaf Forests<br>
         <i style="background:#78d203; width:18px; height:18px; float:left; margin-right:8px;"></i>Deciduous Broadleaf Forests<br>
         <i style="background:#009900; width:18px; height:18px; float:left; margin-right:8px;"></i>Mixed Forests<br>
         <i style="background:#c6b044; width:18px; height:18px; float:left; margin-right:8px;"></i>Closed Shrublands<br>
         <i style="background:#dcd159; width:18px; height:18px; float:left; margin-right:8px;"></i>Open Shrublands<br>
         <i style="background:#dade48; width:18px; height:18px; float:left; margin-right:8px;"></i>Woody Savannas<br>
         <i style="background:#fbff13; width:18px; height:18px; float:left; margin-right:8px;"></i>Savannas<br>
         <i style="background:#b6ff05; width:18px; height:18px; float:left; margin-right:8px;"></i>Grasslands<br>
         <i style="background:#27ff87; width:18px; height:18px; float:left; margin-right:8px;"></i>Permanent Wetlands<br>
         <i style="background:#c24f44; width:18px; height:18px; float:left; margin-right:8px;"></i>Croplands<br>
         <i style="background:#a5a5a5; width:18px; height:18px; float:left; margin-right:8px;"></i>Urban and Built-up Lands<br>
         <i style="background:#ff6d4c; width:18px; height:18px; float:left; margin-right:8px;"></i>Cropland/Natural Vegetation Mosaics<br>
         <i style="background:#69fff8; width:18px; height:18px; float:left; margin-right:8px;"></i>Permanent Snow and Ice<br>
         <i style="background:#f9ffa4; width:18px; height:18px; float:left; margin-right:8px;"></i>Barren<br>
         <i style="background:#1c0dff; width:18px; height:18px; float:left; margin-right:8px;"></i>Water Bodies<br>
     </div>
    '''
    map_object.get_root().html.add_child(folium.Element(legend_html))

# Create ipywidgets for country selection and year selection.
country_dropdown = widgets.Dropdown(
    options=[('Mauritania', 'mauritania'),
             ('Burkina Faso', 'burkinafaso'),
             ('Chad', 'chad'),
             ('Mali', 'mali'),
             ('Niger', 'niger'),
             ('Senegal', 'senegal'),
             ('Sudan', 'sudan')],
    value='mauritania',
    description='Country:'
)

year_slider = widgets.IntSlider(
    value=2013, min=2001, max=2020, step=1, description='Year:'
)

# Output widget for displaying the map.
output = widgets.Output()

# Function to update the map based on the selected country and year.
def update_map(change):
    # Get current selections.
    country = country_dropdown.value
    year = year_slider.value
    params = countries[country]
    center = params['center']
    zoom = params['zoom']
    geojson_path = params['geojson']
    
    # Create a new Folium map centered on the selected country.
    new_map = folium.Map(location=center, zoom_start=zoom)
    folium.TileLayer('OpenStreetMap').add_to(new_map)
    
    # Add the GeoJSON layer for the country's boundary.
    folium.GeoJson(
        geojson_path,
        name=f"{country.title()} Boundary"
    ).add_to(new_map)
    
    # Retrieve the MODIS image for the selected year.
    image = get_modis_image(year)
    new_map.add_ee_layer(image, vis_params, 'MODIS Land Cover ' + str(year))
    
    # Add the custom legend.
    add_legend(new_map)
    
    # Add layer control to toggle layers.
    new_map.add_child(folium.LayerControl())
    
    # Display the updated map.
    with output:
        output.clear_output()
        display(new_map)

# Observe changes in both widgets.
country_dropdown.observe(update_map, names='value')
year_slider.observe(update_map, names='value')

# Display the initial map.
update_map({'new': country_dropdown.value})
display(widgets.VBox([country_dropdown, year_slider, output]))